In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./tmdb_5000_movies.csv")
df.head()

budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3  [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4  [{"id": 818, "name": "based on novel"}, {"id":...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   
2  A cryptic message from Bond’s past sends him o...  107.376788   
3  Following the death of District Attorney Harve...  112.312950   
4  John Carter is a war-weary, former military ca...   43.926995   

                                production_companies  \
0  [{"name": "Ingenious Film Partners", "id": 289...   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...   
2  [{"name": "Columbia Pictures", "id": 5}, {"nam...   
3  [{"name": "Legendary Pictures", "id": 923}, {"...   
4        [{"name": "Walt Disney Pictures", "id": 2}]   

                                production_countries release_date     revenue  \
0  [{"iso_3166_1": "US", "name": "United States o...   2009-12-10  2787965087   
1  [{"iso_3166_1": "US", "name": "United States o...   2007-05-19   961000000   
2  [{"iso_3166_1": "GB", "name": "United Kingdom"...   2015-10-26   880674609   
3  [{"iso_3166_1": "US", "name": "United States o...   2012-07-16  1084939099   
4  [{"iso_3166_1": "US", "name": "United States o...   2012-03-07   284139100   

   runtime                                   spoken_languages    status  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   
1    169.0           [{"iso_639_1": "en", "name": "English"}]  Released   
2    148.0  [{"iso_639_1": "fr", "name": "Fran\u00e7ais"},...  Released   
3    165.0           [{"iso_639_1": "en", "name": "English"}]  Released   
4    132.0           [{"iso_639_1": "en", "name": "English"}]  Released   

                                          tagline  \
0                     Enter the World of Pandora.   
1  At the end of the world, the adventure begins.   
2                           A Plan No One Escapes   
3                                 The Legend Ends   
4            Lost in our world, found in another.   

                                      title  vote_average  vote_count  
0                                    Avatar           7.2       11800  
1  Pirates of the Caribbean: At World's End           6.9        4500  
2                                   Spectre           6.3        4466  
3                     The Dark Knight Rises           7.6        9106  
4                               John Carter           6.1        2124

In [3]:
df["genres"][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [4]:
#장르 전처리
from ast import literal_eval

df["genres"] = df["genres"].apply(literal_eval)
df["keywords"] = df["keywords"].apply(literal_eval)
df["genres"] = df["genres"].apply(lambda x : [y["name"]for y in x])
df["keywords"] = df["keywords"].apply(lambda x : [y["name"]for y in x])
df[["genres", "keywords"]][:1]

genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   

                                            keywords  
0  [culture clash, future, space war, space colon...

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환한다
df["genres_literal"] = df["genres"].apply(lambda x : (" ").join(x))
count_vect = CountVectorizer(min_df = 0, ngram_range = (1, 2))
genre_mat = count_vect.fit_transform(df["genres_literal"])
print(genre_mat.shape)

(4803, 276)


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [7]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


In [8]:
#특정 영화와 장르 유사도가 높은 영화를 반환하는 함수
def find_sim_movie(df, sorted_ind, title_name, top_n = 10) :
  #인자로 입력된 df dataframe에서 "title"컬럼이 입력된 title_name 값인 dataframe추출
  title_movie = df[df["title"] == title_name]

  #title_name을 가진 dataframe의 index 객체를 ndarray로 반환한 후
  #sorted_ind 인자로 입력된 genre_sim_ind 객체에서 유사도 순으로 top_n개 index추출

  title_index = title_movie.index.values
  similar_indexs = sorted_ind[title_index, :(top_n)]

  #추출된 top_n index들 출력, top_n index는 2차원 데이터
  #dataframe에서 index로 사용하기 위해 1차원 array로 변경

  print(similar_indexs)
  similar_indexs = similar_indexs.reshape(-1)

  return df.iloc[similar_indexs]

In [10]:
similar_movies = find_sim_movie(df, genre_sim_sorted_ind, "The Godfather", 15)
similar_movies[["title", "vote_average"]]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866 3112 4041  588 3337
  3378]]


title  vote_average
2731                          The Godfather: Part II           8.3
1243                                    Mean Streets           7.2
3636                                   Light Sleeper           5.7
1946  The Bad Lieutenant: Port of Call - New Orleans           6.0
2640         Things to Do in Denver When You're Dead           6.7
4065                                      Mi America           0.0
1847                                      GoodFellas           8.2
4217                                            Kids           6.8
883                              Catch Me If You Can           7.7
3866                                     City of God           8.1
3112                         Blood Done Sign My Name           6.0
4041                                 This Is England           7.4
588                  Wall Street: Money Never Sleeps           5.8
3337                                   The Godfather           8.4
3378                                      Auto Focus           6.1

In [17]:
# @가증평점 (weighted ratin) :
# (v/(v+m)) * R + (m/(v+m)) * C

# v : 영화별 평점을 투표한 횟수(vote_count)
# m : 평점을 부여하기 위한 최소 투표 횟수, 여기서는 투표수 상위 70%
# R : 개별 영화에 대한 평균 평점(vote_average)
# C : 전체 영화에 대한 평균 평점(df["vote_average"].mean())
# C, m 은 고정값
# v, R 은 영화마다 변동값

# 상위 60%에 해당하는 vote_count를 최소 투표 횟수인 m으로 지정
C = df["vote_average"].mean()
m = df["vote_count"].quantile(0.7)

# C : 전체 영화에 대한 평균 평점 = 약 6점
# m : 평점을 부여하기 위한 최소 투표 횟수 = 370회(상위 70% 수준)
print("C : ", round(C, 2), "m : ", round(m, 2))

C :  6.09 m :  581.0


In [18]:
def weighted_vote_average(record) :
    v = record["vote_count"]
    R = record["vote_average"]

    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )

In [19]:
# 기존 데이터에 가중평점 칼럼 추가
df["weighted_vote"] = df.apply(weighted_vote_average, axis = 1)

In [20]:
def find_sim_movie_ver2(df, sorted_ind, title_name, top_n=15) :
    title_movie = df[df["title"] == title_name]
    title_index = title_movie.index.values

    # top_n 의 2배에 해당하는 장르 유사성이 높은 index 추출
    similar_indexes = sorted_ind[title_index, :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]

    # top_n 의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출
    return df.iloc[similar_indexes].sort_values("weighted_vote", ascending = False)[:top_n]

In [21]:
similar_movies = find_sim_movie_ver2(df, genre_sim_sorted_ind, "The Godfather", 15)
similar_movies[["title", "vote_average", "weighted_vote", "genres", "vote_count"]]

title  vote_average  weighted_vote  \
1881     The Shawshank Redemption           8.5       8.340775   
2731       The Godfather: Part II           8.3       7.972685   
1847                   GoodFellas           8.2       7.869817   
3866                  City of God           8.1       7.612923   
3887                Trainspotting           7.8       7.493372   
883           Catch Me If You Can           7.7       7.486529   
1663  Once Upon a Time in America           8.2       7.457789   
892                        Casino           7.8       7.274445   
281             American Gangster           7.4       7.035214   
1149              American Hustle           6.8       6.678616   
2582   The Place Beyond the Pines           6.8       6.615914   
4041              This Is England           7.4       6.595076   
1243                 Mean Streets           7.2       6.504915   
2839                     Rounders           6.9       6.439855   
1370                           21           6.5       6.378861   

              genres  vote_count  
1881  [Drama, Crime]        8205  
2731  [Drama, Crime]        3338  
1847  [Drama, Crime]        3128  
3866  [Drama, Crime]        1814  
3887  [Drama, Crime]        2655  
883   [Drama, Crime]        3795  
1663  [Drama, Crime]        1069  
892   [Drama, Crime]        1307  
281   [Drama, Crime]        1502  
1149  [Drama, Crime]        2807  
2582  [Drama, Crime]        1653  
4041  [Drama, Crime]         363  
1243  [Drama, Crime]         345  
2839  [Drama, Crime]         439  
1370  [Drama, Crime]        1375